# 0. 기본 정보
* 코드 작성자: 여서연
* 코드 작성일: 2024-05-06 ~ 
* 데이터 분석 목적: 크롤링 대상 적합성 판정

# 1. 기초 설정

In [1]:
# 사용한 라이브러리

from googleapiclient.discovery import build
import pandas as pd

from dotenv import load_dotenv
import os

import warnings
warnings.filterwarnings('ignore')

# 2. 유튜브 댓글 크롤링
* [코드 출처](https://velog.io/@jihyunko/python-%EC%9C%A0%ED%8A%9C%EB%B8%8C-%EB%8C%93%EA%B8%80-%ED%81%AC%EB%A1%A4%EB%A7%81)
* [참조](https://developers.google.com/youtube/v3/docs/commentThreads/list?hl=ko)

## 함수 소개
* YoutubeCommentCrawl(key, url, max=100)
    * key: Google Cloud API에서 발급 받은 API 키
    * url: 댓글을 크롤링할 주소 (단, 상단바에서 복사할 것)
    * max: 결과 집합에 반환해야 하는 최대 항목 수 (1 ~ 100)

In [2]:
load_dotenv()
myKEY = os.environ.get('GOOGLE_API_KEY')

url1 = 'https://www.youtube.com/watch?v=rh73ASuF038'
url2 = 'https://www.youtube.com/watch?v=SX3o62ek70s'

In [3]:
def YoutubeCommentCrawl(key, url, max=100):
    video_id = url[-11:] #영상 주소에서 추출
    comments = list() #댓글을 담을 리스트
    api_obj = build('youtube', 'v3', developerKey=key) #API name, version, key
    response = api_obj.commentThreads().list(part='snippet, replies', videoId=video_id, maxResults=max).execute()

    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])

            if item['snippet']['totalReplyCount'] > 0:
                for reply_item in item['replies']['comments']:
                    reply = reply_item['snippet']
                    comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])

        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet,replies', videoId='rh73ASuF038', pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break

    df = pd.DataFrame(comments, columns=['comment', 'author', 'date', 'num_likes'])
    return df

In [4]:
df1 = YoutubeCommentCrawl(myKEY, url1)
df2 = YoutubeCommentCrawl(myKEY, url2)

In [5]:
df1.sample()

,comment,author,date,num_likes
311,저정도면 국가차원에서 뭘 막아야하는거 아님?,@user-qk6ok4zx7b,2023-04-12T09:07:38Z,81


In [6]:
df2.sample()

,comment,author,date,num_likes
36,지금 우리는 누구의 책임이 문제가 아니라 해결책이 가장 우선이다,@user-qn1yc8up5w,2019-02-09T01:06:01Z,0


# 3. NLP